In [1]:
import csv
import json
import re
import ast

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
def getItem(category):
    if category is None:
        url = 'https://api.nookplaza.net/items'
    else:
        url = 'https://api.nookplaza.net/items?category={}'.format(category)
    r = requests.get(url)
    text = r.json()
    print('get {} {}'.format(category, len(text['results'])))
    return text['results']

In [3]:
def saveFile(category, output, dictName):
    with open('../../nook_link/src/data/{}.json'.format(category), 'wt') as fout:
        fout.write(json.dumps(output))

    with open('{}.json'.format(category), 'wt') as fout:
        fout.write(json.dumps(output))

    with open('{}.csv'.format(category), 'wt', encoding='utf-8', newline='') as fout:
        csvout = csv.writer(fout)

        output_csv = [dictName]
        for row in output:
            row_ = [row[name] for name in dictName]
            output_csv.append(row_)
        csvout.writerows(output_csv)
    
    print('save {} over'.format(category))

In [4]:
def getEn2twFn():
    nookplazaUrl = 'https://nookplaza.net'
    r = requests.get(nookplazaUrl)

    m = re.search('main\.dart\.js\?v=(\d*)', r.text)
    dartJs = m.group()

    mainJsUrl = '{}/{}'.format(nookplazaUrl, dartJs)
    r = requests.get(mainJsUrl)
    mainJs = r.text

    # 'C.Iz=new H.b(13,{},C.a,[P.a,P.a])'
    m = re.findall('new H\.b\(13,(.*),C\.a,\[P\.a,P\.a\]\)', mainJs)
    m = [item.encode("utf-8").decode('unicode-escape') for item in m]
    m = [ast.literal_eval(item) for item in m if not ':null' in item]

    nameMap = {}
    for item in m:
        nameMap[item["en-us"]] = item

    print('get {} i18n data'.format(len(nameMap.keys())))

    try:
        df = pd.read_excel('../tempData/Furniture.xlsx', sheetname=0)
    except:
        df = pd.read_excel('../tempData/Furniture.xlsx', sheet_name=0)

    EnTwTable = ['類別', '取得方式', '標籤', '主題', '款式', '樣式', '款式說明', '樣式說明', 'DIY相關', '藝術品', '作者/年代/材質', '藝術品真實圖', '村民']

    nameMapCustom = {}
    for EnTw in EnTwTable:
        itemType_en = df[EnTw].tolist()
        itemType_tw = df['{}_tw'.format(EnTw)].tolist()
        itemType_en = [x for x in itemType_en if str(x) != 'nan']
        itemType_tw = [x for x in itemType_tw if str(x) != 'nan']

        for item in zip(itemType_en, itemType_tw):
            if item[0] in nameMapCustom and item[1] != nameMapCustom[item[0]]['zh-tw']:
                print(item)
                print(nameMapCustom[item[0]])
                print('------\n')

            nameMapCustom[item[0]] = {
                'zh-tw': item[1]
            }

    print('Custom {} i18n data'.format(len(nameMapCustom.keys())))

    def en2twFn(en, Custom = False):
        if Custom == 'onlySite':
            return nameMap[en]

        if Custom:
            if en in nameMapCustom:
                return nameMapCustom[en]

            elif en.lower() in nameMapCustom:
                return nameMapCustom[en.lower()]

            elif (en[0].lower() + en[1:]) in nameMapCustom:
                return nameMapCustom[(en[0].lower() + en[1:])]

        if en.lower() in nameMap:
            return nameMap[en.lower()]

        elif (en[0].lower() + en[1:]) in nameMap:
            return nameMap[(en[0].lower() + en[1:])]

        elif en in nameMap:
            return nameMap[en]

        elif en in nameMapCustom:
                return nameMapCustom[en]

        elif en.lower() in nameMapCustom:
            return nameMapCustom[en.lower()]

        elif (en[0].lower() + en[1:]) in nameMapCustom:
            return nameMapCustom[(en[0].lower() + en[1:])]

        elif en.replace('The', 'the') in nameMap:
            return nameMap[en.replace('The', 'the')]

        elif en.replace('K.K. Tude', 'K.K. Étude') in nameMap:
            return nameMap[en.replace('K.K. Tude', 'K.K. Étude')]

        elif en.replace('Caf K.K.', 'Café K.K.') in nameMap:
            return nameMap[en.replace('Caf K.K.', 'Café K.K.')]

        print(en)
        raise en
        
    return en2twFn

In [5]:
en2twFn = getEn2twFn()

idMap = {}
name2fileNameMap = {}
allItem = getItem(None)
for item in allItem:
#     if 'ifier' in item['name']:
#         print(item)
        
    if item['name'].startswith("Present ("):
        continue
        
    if 'internalID' in item['content']:
        try:
            name = en2twFn(item['name'].replace(' (fake)', ''))['zh-tw']
            idMap[item['content']['internalID']] = {'name': name, 'filename': item['content']['image']}
            name2fileNameMap[item['name']] = item['content']['filename']
        except:
            pass
#             print(item['name'])

print('get {} id to Name data'.format(len(idMap.keys())))

get 6047 i18n data
Custom 1200 i18n data
get None 5845
Bell bag
Turnips
Coin
get 4589 id to Name data


In [10]:
en2twFn('Pacifier')

# nameMap['Pacifier']
# name2fileNameMap['Stormy-night wall']

{'en-us': 'pacifier',
 'en-gb': 'pacifier',
 'de-de': 'Schnullers',
 'es-es': 'chupete',
 'fr-fr': 'tétine',
 'fr-ca': 'suce',
 'it-it': 'ciuccio',
 'nl-nl': 'speen',
 'zh-cn': '奶嘴',
 'zh-tw': '奶嘴',
 'ja-jp': 'おしゃぶり',
 'ko-kr': '노리개젖꼭지',
 'ru-ru': 'соска-пустышка'}

In [6]:
en2twFn = getEn2twFn()
output = []
furnitures = getItem('Furniture')
recipes = getItem('Recipes')

DIYMap = {}
for diy in recipes:
    DIYMap[diy['name']] = diy['content']

for item in furnitures:
    if item["category"] == "Art":
        continue

    pickData = {'type': 'furnitures'}
    name = en2twFn(item['name'])
    
#     print(name['zh-tw'])
    
    pickData['id'] = int(item['content']['internalID'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']

    pickData['buy'] = item['content']['buy'] if 'buy' in item['content'] else None
    pickData['sell'] = item['content']['sell']

    pickData['category'] = en2twFn(item['content']['category'], True)['zh-tw']
    pickData['tag'] = en2twFn(item['content']['tag'], True)['zh-tw']
    pickData['themes'] = [en2twFn(theme, True)['zh-tw'] for theme in item['content']['themes'] if theme != 'None']
    
    pickData['interact'] = item['content']['interact']
    pickData['size'] = '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])
    pickData['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickData['DIY'] = item['content']['dIY']
#     pickData['diyInfo'] = {}
    pickData['diyInfoMaterials'] = []
    pickData['diyInfoObtainedFrom'] = []
    pickData['diyInfoSourceNotes'] = ''

    if item['content']['dIY']:
        diyInfo = DIYMap[item['name']]
        pickData['diyInfoMaterials'] = [{'count': item['count'], 'itemName': en2twFn(item['itemName'])['zh-tw']} for item in diyInfo['materials']]
        pickData['diyInfoObtainedFrom'] = [en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in diyInfo['obtainedFrom']]
        
        try:
            pickData['diyInfoSourceNotes'] = '' if diyInfo['sourceNotes'] == '' else en2twFn(diyInfo['sourceNotes'], True)['zh-tw']
        except:
            pickData['diyInfoSourceNotes'] = diyInfo['sourceNotes']
        
    pickData['filename'] = item['content']['filename']
    
    if 'bodyTitle' in item['content'] and item['content']['bodyTitle']:
        pickData['bodyTitle'] = en2twFn(item['content']['bodyTitle'], True)['zh-tw']
    else:
        pickData['bodyTitle'] = '無'
        
    if 'bodyCustomize' in item['content']:
        pickData['bodyCustomize'] = item['content']['bodyCustomize']
    else:
        pickData['bodyCustomize'] = False
    
    if 'patternTitle' in item['content'] and item['content']['patternTitle']:
        pickData['patternTitle'] = en2twFn(item['content']['patternTitle'], True)['zh-tw']
    else:
        pickData['patternTitle'] = '無'
        
    if 'patternCustomize' in item['content']:
        pickData['patternCustomize'] = item['content']['patternCustomize']
    else:
        pickData['patternCustomize'] = False

    variations = {
        "bodys": {},
        "pattrens": {}
    }
    for variation in item['variations']:
        content = variation['content']
        bodyIndex, pattrenIndex = content['filename'].split('_')[-2:]
        
        bodyColor = en2twFn(content['bodyColor'], True)['zh-tw'] if 'bodyColor' in content and content['bodyColor'] is not None else ''
        patternColor = en2twFn(content['patternColor'], True)['zh-tw'] if 'patternColor' in content and content['patternColor'] is not None else ''
        
        variations['bodys'][bodyColor] = bodyIndex
        variations['pattrens'][patternColor] = pattrenIndex
        
    pickData['variations'] = variations
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])

dictName = ['id', 'name_e', 'name_j', 'name_c', 'buy', 'sell', 'category', 'tag', 'themes', 'interact', 'size', 'obtainedFrom', 'DIY', 'diyInfoMaterials', 'diyInfoObtainedFrom', 'diyInfoSourceNotes']
dictName += ['filename', 'bodyTitle', 'patternTitle', 'bodyCustomize', 'patternCustomize', 'variations']

saveFile('furniture', output, dictName)

get 6047 i18n data
Custom 1199 i18n data
get Furniture 1131
get Recipes 609
Obtained during the main storyline during the villager house development quest if the native fruit is oranges
50% chance of being obtained during the main storyline during the villager house development quest
50% chance of being obtained during the main storyline during the villager house development quest
Fish up 3 empty cans
Obtained during the main storyline during the villager house development quest if the native fruit is peaches
Purchase Test Your DIY Skills from Nook's Cranny
Obtained during the main storyline during the villager house development quest if the native fruit is pears
50% chance of being obtained during the main storyline during the villager house development quest
Purchase Test Your DIY Skills from Nook's Cranny
Obtained during the main storyline during the villager house development quest if the native fruit is pears
Purchase Test Your DIY Skills from Nook's Cranny; Obtained during the ma

In [14]:
artInfo = [
    ["art_00002","雪中獵人 老彼得‧布勒哲爾 1565年 [油彩‧木板]"],
    ["art_00002_1","布勒哲爾是文藝復興後期的風景畫家，多以農民生活為題材，因此也有人稱之為「農民畫家」。近處描繪了打獵失敗而疲憊不堪的獵人們，遠處則是在結冰的湖面上開心滑冰的人們，這樣的對比相當耐人尋味。"],
    ["art_00006","回眸美人圖 菱川師宣 17世紀 [絹本著色]"],
    ["art_00006_1","這幅畫為江戶時代前期浮世繪始祖菱川師宣的手繪浮世繪作品，並非版畫。描繪一名優雅自信的女性，身上穿著當時盛行的服飾，回眸一看的瞬間，其姿態極富情感。"],
    ["art_00010","倒牛奶的女僕 約翰尼斯‧維梅爾 1658年左右 [油彩‧畫布]"],
    ["art_00010_1","維梅爾擅長以精準的構圖和巧妙的光影來表現，因此又被稱為「光的畫家」。此為其25歲時的作品。沒想到如此年輕。"],
    ["art_00014","穿藍衣的少年 托馬斯‧庚斯博羅 1770年 [油彩‧畫布]"],
    ["art_00014_1","庚斯博羅留有多幅肖像畫，經常使用有別於傳統肖像畫的新穎顏色。《穿藍衣的少年》為其中之一。庚斯博羅說過「肖像畫是為了賺錢，風景畫才是愛好」，但諷刺的是，其肖像畫對近代英國繪畫卻有極大的影響。"],
    ["art_00018","蒙娜麗莎 李奧納多‧達文西 1503年左右 [油彩‧木板]"],
    ["art_00018_1","世界上最有名的微笑。達文西直到臨終前都不願放手的名作。直到現在，仍有一大群人想找出並解開這幅畫的種種謎團。可說是相當具有魅力的畫作。"],
    ["art_00020","蘋果和柳橙的靜物 保羅‧塞尚 1899年左右 [油彩‧畫布]"],
    ["art_00020_1","「近代繪畫之父」塞尚與畢卡索同為對後代影響深遠的畫家。而這幅靜物畫正是塞尚所繪製的。此畫作並非將眼前所見真實地描繪出來，而是從不同視角表現出結構之美。不只讓人有靜物漂浮的感受，同時也描繪出觀賞者的內心，是一幅前所未有的畫作。"],
    ["art_00024","抱銀貂的女子 李奧納多‧達文西 1490年左右 [油彩‧木板]"],
    ["art_00024_1","達文西創作的女性肖像畫，包含《蒙娜麗莎》在內，僅有四幅。這是其中一幅。也有人說抱著的其實不是銀貂，而是白毛的雪貂。"],
    ["art_00028","戴珍珠耳環的少女 約翰尼斯‧維梅爾 1665年左右 [油彩‧畫布]"],
    ["art_00028_1","這幅畫作不惜使用當時要價不斐的群青色顏料畫出格外引人注目的鮮豔藍色，因此也有人稱之為《藍色頭巾的少女》。據說也有人認為那副耳環原本並非真的珍珠。"],
    ["art_00032","維納斯的誕生 山德羅·波提且利 1485年左右 [蛋彩‧畫布]"],
    ["art_00032_1","這幅畫描繪了希臘羅馬神話中的女神維納斯站在貝殼上從海面誕生的姿態。波提且利原本是一個外號，來自他那位胖得跟酒桶一樣的哥哥。不過，為什麼會繼承哥哥的外號，就不得而知了。這是一幅知名的畫作，相信資深設計師對它尤其熟悉。"],
    ["art_00034","著衣的馬哈 法蘭西斯科‧哥雅 1805年左右 [油彩‧畫布]"],
    ["art_00034_1","哥雅是活躍於18世紀後半到19世紀前半的西班牙大畫家。《著衣的馬哈》與另一幅《裸體的馬哈》，兩幅畫是一對的作品。據說當時是為了掩飾而畫的。「馬哈」的意思是「漂亮的馬德里姑娘」，並非人名。"],
    ["art_00038","富嶽三十六景-神奈川沖浪裏 葛飾北齋 1831年左右 [木版畫]"],
    ["art_00038_1","這是江戶時代後期極具代表性的浮世繪畫家葛飾北齋於60歲後開始著手的系列中最具代表性的一幅。原有三十六景，後來又追加了十景，共有四十六景。"],
    ["art_00042","四季「夏季」 朱塞佩·阿爾欽博托 1573年左右 [油彩‧畫布]"],
    ["art_00042_1","這是16世紀後半義大利的奇想畫家阿爾欽博托最擅長的奇幻風格肖像畫。仔細一看會發現這張人的側臉是由水果和蔬菜組合而成的。"],
    ["art_00044","拾穗 尚-法蘭索瓦‧米勒 1857年 [油彩‧畫布]"],
    ["art_00044_1","這是19世紀法國畫家米勒的代表作品。其畫作多以農民貧窮的日常生活為題材。遠處是農場地主和佃農們的收穫情景，而近處則是描繪貧窮的人們撿拾掉落在田地上的麥穗。據說當時會為了貧窮的人們故意在收穫時留下一點麥穗。"],
    ["art_00046","女神遊樂廳的吧檯 愛德華‧馬奈 1882年左右 [油彩‧畫布]"],
    ["art_00046_1","女子身後的大鏡子裡映照出來劇場的人們熱鬧喧囂的樣子。仔細一看，鏡中反照出這名女子的背面，其位置關係和物品配置上有諸多矛盾，暗藏著許多謎團。這是馬奈結合多方視角所構圖而成的最後大作。在此畫完成後的隔一年，馬奈逝世，享年51歲。"],
    ["art_00048","吹笛子的少年 愛德華‧馬奈 1866年 [油彩‧畫布]"],
    ["art_00048_1","這是馬奈初期的作品，描繪一名穿軍服的少年正在吹奏笛子。人物像搭配空無一物的背景，這種構圖方式是受到維拉斯奎茲的影響。馬奈這種毫無遠近感，只以強烈對比的色塊構成的作品，對後代的畫家有很大的影響，同時也被稱之為「印象派之父」。順帶一提，畫中正在吹奏的是橫笛，依手指位置來看，他正在吹的音是「So」。"],
    ["art_00050","向日葵 文森‧威廉‧梵谷 1888年左右 [油彩‧畫布]"],
    ["art_00050_1","這幅畫是梵谷為了追求鮮豔的色彩，移居南法亞爾時所創作的作品。黃色是梵谷最愛的顏色。梵谷在亞爾繪製的向日葵畫作共有七件，現存六件，其中一件在東京。"],
    ["art_00052","播種者 尚-法蘭索瓦‧米勒 1857年 [油彩‧畫布]"],
    ["art_00052_1","這幅畫是從巴黎搬到農村後，描繪農民朝田地撒種的樣子。畫中除了描繪與大地共生的農民強而有力的樣貌外，還包含了宗教的意義。對後代的畫家，包括梵谷，有深遠的影響。"],
    ["art_00056","夜巡 林布蘭‧哈爾曼松‧范‧萊因 1642年 [油彩‧畫布]"],
    ["art_00056_1","這幅畫是17世紀荷蘭畫家林布蘭描繪民兵隊出巡的最高傑作。當時的肖像畫普遍還是直立不動的，但此畫作卻生動地描繪民兵的動作，巧妙地搭配光影，呈現出戲劇化的場景。原本以為這幅畫是描繪黑夜中的情景，直到修復畫作時，才知道應該是白天，只因畫作上的漆發黑，才會看起來暗暗的。"],
    ["art_00066","三世大谷鬼次-奴江戶兵衛 東洲齋寫樂 1794年 [木版畫]"],
    ["art_00066_1","這幅畫為江戶時代中期的浮世繪畫師東洲齋寫樂所畫的演員肖像。寫樂留有140張浮世繪於世，但其活躍期間卻僅約10個月。"],
    ["art_00068","自由引導人民 歐仁‧德拉克羅瓦 1830年 [油彩‧畫布]"],
    ["art_00068_1","19世紀法國浪漫主義代表畫家德拉克羅瓦於1830年經歷法國7月革命後，同年繪製了這幅畫。正中間的女子經常被誤認為「聖女貞德」，但其實她的形象是來自於一位架空的女性「瑪莉安」。"],
    ["art_00072","侍女 迪亞哥‧維拉斯奎茲 1656年左右 [油彩‧畫布]"],
    ["art_00072_1","這是任職於西班牙宮廷的維拉斯奎茲的名作。由於畫出小公主和身旁侍女們的樣子，因此被命名為「侍女」。畫面左側是畫家維拉斯奎茲本人，右側的鏡中則是映照著國王和皇后的身影。其實這幅畫是從國王視角看向維拉斯奎茲在為國王和皇后作畫的情景。"],
    ["art_00076","風神雷神圖屏風（風神） 俵屋宗達 寛永年間左右 [紙本金地著色]"],
    ["art_00076_1","這是江戶時代初期畫家俵屋宗達的大作，在兩扇一對的屏風上貼滿金箔作為背景，以流動感強烈的筆鋒畫上風神和雷神。題材簡略化、形式化和省略背景的設計風格為之後的尾形光琳所繼承，並以「琳派」流傳到現代。"],
    ["art_00078","大碗島的星期天下午 喬治‧秀拉 1885年左右 [油彩‧畫布]"],
    ["art_00078_1","這幅畫是新印象派創始人秀拉僅以小圓點所創作的作品。秀拉還發明了為呈現鮮明的色彩而不將顏料混色的技法。這幅人們在法國塞納河中的一座小島上開心度假的情景，費時兩年才完成。"],
    ["art_01331","母狼乳嬰 作者不明 西元前５世紀左右 [青銅]"],
    ["art_01331_1","這是羅馬神話裡養育雙胞胎的狼。雙胞胎是於15世紀時才加上去的，而狼的銅像則是西元前5世紀的歷史性傑作。話雖如此，但最近又有研究顯示這是11～12世紀的銅像。（本館的展示暫且保留西元前5世紀左右的說法。）"],
    ["art_01333","大衛像 米開朗基羅‧博納羅蒂 1504年左右 [大理石]"],
    ["art_01333_1","這是雙眼怒視敵人，手拿布條，準備朝敵人投出石頭的少年大衛像。米開朗基羅費時三年多才製作完成。仔細看眼睛會發現瞳孔被刻成愛心的形狀，但因為當時還沒有愛心符號的概念，所以被認為是一種光線映入瞳孔的表現。不過，也太大了吧。"],
    ["art_01335","擲鐵餅者 作者不明 2世紀左右 [大理石]"],
    ["art_01335_1","這是以古希臘雕刻家米隆的作品為原型，於羅馬製作的雕像。是一個展現人體美的作品。無論從哪個角度觀看都很美。"],
    ["art_01337","遮光器土偶 作者不明 西元前1000年～400年左右 [素燒陶器]"],
    ["art_01337_1","這是繩文時代出土的神秘人形土偶，以泥土捏製並素燒而成。大大圓圓的眼睛猶如可以遮光的護目鏡，因此被叫做「遮光器」土偶。"],
    ["art_01339","卡美哈梅哈大帝像 湯瑪斯‧R‧古爾德 1883年 [青銅]"],
    ["art_01339_1","這是1810年夏威夷統一時，建立夏威夷王朝的卡美哈梅哈大帝的銅像。每年6月11日卡美哈梅哈大帝節時，上面都會被掛上很多花圈。"],
    ["art_01341","米洛的維納斯 作者不明 西元前130年左右 [大理石]"],
    ["art_01341_1","這座愛與美的女神維納斯的石像是在米洛島上發現的。這座美麗的雕塑究竟本來是什麼姿勢，引起大家無限想像。"],
    ["art_01343","娜芙蒂蒂胸像 圖特摩斯 西元前1345年左右 [石灰岩]"],
    ["art_01343_1","這是古埃及三大美女之一娜芙蒂蒂的胸像，其名字含有「難得一見的美人」之意。左眼的缺陷究竟是嵌入物被人取出，還是尚未完成，目前仍不得而知。"],
    ["art_01345","薩莫色雷斯的勝利女神 作者不明 西元前190年左右 [大理石]"],
    ["art_01345_1","這是在薩莫色雷斯島上發現的勝利女神尼克的雕像。呈現出女神降落在船頭的樣子。雕像被發現時支離破碎，後來才組合復原，不過頭部和兩隻手仍尚未找到。"],
    ["art_12533","奧爾梅克巨型頭顱 作者不明 西元前1000年左右 [玄武岩、安山岩]"],
    ["art_12533_1","這是在墨西哥的奧爾梅克古文明遺跡中發現的巨大石像。因此被稱為奧爾梅克頭顱。最大的高達3公尺。一般認為當時只建造了頭部，沒有軀體。"],
    ["art_12535","羅塞塔石碑 作者不明 西元前196年 [花崗閃長岩]"],
    ["art_12535_1","這是1799年於埃及羅塞塔發現的石碑。是解讀古埃及神聖文字（一種象形文字）的重要線索。類似的文章以三種文字記載，上段為神聖文字聖書體，中段為平民文字世俗體，下段為古希臘文。"],
    ["art_12537","后母戊鼎 作者不明 西元前1200年 [青銅]"],
    ["art_12537_1","這是中國古代最大且最重的祭祀青銅器。鼎的內壁刻有銘文，因此判定是君王為祭祀母親而鑄造的。"],
    ["art_12539","兵馬俑 作者不明 西元前210年 [陶]"],
    ["art_12539_1","這座素燒的士兵像是為了在死後的世界守護秦始皇而建造的陪葬品。1974年發現了將近8000件歷經2000多年的雕像。每座雕像都與真人大小相同，並且不是以模具製作，而是人工雕刻的，因此不論表情、服裝還是姿勢，都有所不同。"],
    ["art_12541","沉思者 奧古斯特‧羅丹 1902年 [青銅]"],
    ["art_12541_1","這座舉世聞名的銅像是近代雕刻之父，法國雕刻家羅丹建造的。原雕塑位於法國，被鑄造超過20件，所以世界各地的展示品都被視為真品。在日本的上野和京都也都有戶外展示。"],
    ["art_12570","風神雷神圖屏風（雷神） 俵屋宗達 寛永年間左右 [紙本金地著色]"],
    ["art_12570_1","這是江戶時代初期畫家俵屋宗達的大作，在兩扇一對的屏風上貼滿金箔作為背景，以流動感強烈的筆鋒畫上風神和雷神。題材簡略化、形式化和省略背景的設計風格為之後的尾形光琳所繼承，並以「琳派」流傳到現代。"],
    ["art_12618","星夜 文森‧威廉‧梵谷 1889年左右 [油彩‧畫布]"],
    ["art_12618_1","這是梵谷在精神病院療養時所繪製的晚年代表作。但這並不是真實呈現從病房看到的風景，而是包含了記憶和幻想。梵谷在寫信給其他畫家時提到這幅畫是一件失敗的作品，可見梵谷當時對自己相當不滿意。"],
    ["art_12619","維特魯威人 李奧納多‧達文西 1487年左右 [筆與墨水‧紙張]"],
    ["art_12619_1","這幅人體圖是參考西元前１世紀的建築師 維特魯威的著作《建築書》後，按照人體的理想比例所繪製出來的。"],
    ["art_12621","奧菲莉亞 約翰‧艾佛雷特‧米萊 1852年 [油彩‧畫布]"],
    ["art_12621_1","奧菲莉亞是莎士比亞名劇「哈姆雷特」中的女主角。在描繪得精美且細緻的大自然中，女子如失了魂般倒在河川裡，臉上帶著看破生死的莊嚴表情，牽動著鑑賞者的心。"],
    ["art_12622","紫陽花雙雞圖 伊藤若沖 18世紀 [絹本著色]"],
    ["art_12622_1","這幅畫是由江戶時代中期活躍於京都的奇想畫家伊藤若沖所創作的雞圖。畫中色彩鮮艷的雞相當精緻且栩栩如生。200多年前的作品至今仍然色彩鮮艷，據說其中一個原因是因為採用了極高品質（昂貴）的礦石顏料和畫布。"],
    ["art_12624","勇莽號戰艦 約瑟夫‧瑪羅德‧威廉‧特納 1839年 [油彩‧畫布]"],
    ["art_12624_1","這是光的畫家特納的名作。由納爾遜將軍率領的英國海軍戰艦，在完成最後的任務後，被拖向拆船廠的情景被描繪得栩栩如生。"],
    ["art_12625","死之島 阿諾德‧勃克林 1883年 [油彩‧畫布]"],
    ["art_12625_1","這座墳墓之島漂浮在如鏡子般幽靜的海上，四周環繞著懸崖峭壁，而承載屍體的小船無聲地靠近。整幅畫帶有恐怖卻平靜的氣氛。這幅畫是象徵主義畫家勃克林的作品，並非真實描繪眼前所見景象，而是想要呈現出人類隱藏的內心和想法。"]
]

artInfoMap = {}
for info in artInfo:
    if not info[0].endswith("_1"):
        name, *artist = info[1].split(' ')
        artInfoMap[int(info[0].replace("art_", ""))] = {
            "name": name,
            "artist": " ".join(artist)
        }
    else:
        artInfoMap[int(info[0].replace("art_", "").replace("_1", ""))]["museumDescription"] = info[1]

print(artInfoMap[12625])
        
furnitures = getItem('Museum')

ArtMap = {}
for item in furnitures:
    if item["category"] != "Art":
        continue
        
    pickArt = {'type': 'arts'}
    name = en2twFn(item['name'].replace(' (fake)', ''))
    
    print(name['zh-tw'])
    if item['content']['internalID'] in artInfoMap:
        artInfo = artInfoMap[item['content']['internalID']]
    else:
        artInfo = {
            "name": "",
            "artist": "",
            "museumDescription": ""
        }

    pickArt['id'] = int(item['content']['internalID'])
    pickArt['category'] = en2twFn(item['category'], True)['zh-tw']
    pickArt['name_c'] = name['zh-tw']
    pickArt['name_j'] = name['ja-jp']
    pickArt['name_e'] = name['en-us']
    
    pickArt['artist'] = artInfo['artist']
    pickArt['filename'] = item['content']['filename']
    pickArt['interact'] = item['content']['interact']
    pickArt['itemCategory'] = item['content']['itemCategory']
    pickArt['museumDescription'] = artInfo['museumDescription']
    pickArt['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickArt['realArtworkTitle'] = item['content']['realArtworkTitle']
    pickArt['realArtworkTitle_tw'] = artInfo['name']
    pickArt['tag'] = en2twFn(item['content']['tag'], True)['zh-tw']
    pickArt['themes'] = [en2twFn(theme, True)['zh-tw'] for theme in item['content']['themes'] if theme != 'None']
    pickArt['size'] = '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])
    pickArt['realArtImage'] = en2twFn(item['content']['filename'].replace('Fake', ''))['zh-tw']
    
    if not name['en-us'] in ArtMap:
        ArtMap[name['en-us']] = {}
        
    ArtMap[name['en-us']][str(item['content']['genuine'])] = pickArt
    
artOutput = []
for name in ArtMap.keys():
    if 'False' in ArtMap[name]:
        ArtMap[name]['True']['haveFake'] = True
    else:
        ArtMap[name]['True']['haveFake'] = False
    artOutput.append(ArtMap[name]['True'])

artOutput = sorted(artOutput, key=lambda k: k['id'])

dictName = ['id', 'name_e', 'name_j', 'name_c', 'realArtworkTitle', 'realArtworkTitle_tw', 'artist', 'museumDescription']
dictName += ['obtainedFrom', 'filename', 'haveFake', 'interact', 'itemCategory', 'themes', 'tag', 'size', 'realArtImage']

saveFile('art', artOutput, dictName)

{'name': '死之島', 'artist': '阿諾德‧勃克林 1883年 [油彩‧畫布]', 'museumDescription': '這座墳墓之島漂浮在如鏡子般幽靜的海上，四周環繞著懸崖峭壁，而承載屍體的小船無聲地靠近。整幅畫帶有恐怖卻平靜的氣氛。這幅畫是象徵主義畫家勃克林的作品，並非真實描繪眼前所見景象，而是想要呈現出人類隱藏的內心和想法。'}
get Museum 343
名貴的名畫
粗野的名畫左半邊
舒適的名畫
出乎意料的雕塑
肅穆的名畫
沉沒的名畫
石頭顱雕塑
強壯的雕塑
熱鬧的名畫
很好的名畫
神祕的雕塑
動人的名畫
有趣的名畫
線索的雕塑
端莊的名畫
有名的名畫
磅礡的名畫
細緻的名畫
驚人的名畫
驚人的名畫
學術性的名畫
武士的雕刻
優美的名畫
俊俏的名畫
石頭顱雕塑
強壯的雕塑
平靜的名畫
充滿母愛的雕塑
充滿母愛的雕塑
英挺的雕塑
珍貴的名畫
美麗的雕塑
勇敢的名畫
遠古的雕塑
粗野的名畫右半邊
粗野的名畫左半邊
神聖的雕塑
閃爍的名畫
婀娜的名畫
神祕的雕塑
動人的名畫
端莊的名畫
漂亮的名畫
肅穆的名畫
婀娜的名畫
優美的名畫
俊俏的名畫
平靜的名畫
厲害的名畫
神秘的名畫
強健的名畫
有趣的名畫
線索的雕塑
偉大的雕塑
光線的名畫
英挺的雕塑
有名的名畫
似曾相識的雕塑
細緻的名畫
常見的名畫
和煦的名畫
美麗的雕塑
勇敢的名畫
遠古的雕塑
漂亮的名畫
粗野的名畫右半邊
武士的雕刻
神聖的雕塑
出乎意料的雕塑
學術性的名畫
save art over


In [7]:
output = []

recipes = getItem('Recipes')
allItem = getItem(None)

sellMap = {}
for item in allItem:
    if 'sell' in item['content'] and item['category'] != 'Recipes':
        sellMap[item['name']] = item['content']['sell']

for item in recipes:

#     print(item['name'])
#     print('')

    if 'construction kit' in item['name']:
        continue
    pickData = {'type': 'recipes'}
    name = en2twFn(item['name'])

    pickData['id'] = int(item['content']['internalID'])
#     print(name['zh-tw'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']
    
    pickData['category'] = en2twFn(item['content']['itemCategory'], True)['zh-tw']
    pickData['filename'] = item['content']['itemImage'].split('/')[-1].replace('.png', '')
    pickData['DIY'] = True
#     pickData['sell'] = item['content']['sell']
    pickData['sell'] = sellMap[item['name']]
    pickData['materials'] = [{'count': item['count'], 'itemName': en2twFn(item['itemName'])['zh-tw']} for item in item['content']['materials']]
    pickData['obtainedFrom'] = [en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in item['content']['obtainedFrom']]
#     pickData['sourceNotes'] = '' if item['content']['sourceNotes'] == '' else en2twFn(item['content']['sourceNotes'], True)['zh-tw']
    
    try:
        pickData['sourceNotes'] = '' if item['content']['sourceNotes'] == '' else en2twFn(item['content']['sourceNotes'], True)['zh-tw']
    except:
        pickData['sourceNotes'] = item['content']['sourceNotes']
    
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])
dictName = ['id', 'name_e', 'name_j', 'name_c', 'category', 'filename', 'sell', 'materials', 'obtainedFrom', 'sourceNotes']

saveFile('recipes', output, dictName)

get Recipes 609
get None 5845
Obtained during the main storyline during the villager house development quest if the native fruit is apples
Purchase Test Your DIY Skills from Nook's Cranny; Obtained during the main storyline during the villager house development quest
Obtained during the main storyline during the villager house development quest if the native flower is roses
Obtained during the main storyline during the villager house development quest if the native flower is pansies
Obtained during the main storyline during the villager house development quest if the native fruit is oranges
Obtained during the main storyline during the villager house development quest if the native flower is mums
Obtained during the main storyline during the villager house development quest if the native fruit is cherries
Obtained during the main storyline during the villager house development quest if the native flower is windflowers
50% chance of being obtained during the main storyline during the vi

In [ ]:
output = []

villagers = getItem('Villagers')

for item in villagers:
    if item['category'] != "Villagers":
        continue

#     print(item['content'])
    pickData = {'type': 'villagers'}
    name = en2twFn(item['name'][0].upper() + item['name'][1:], 'onlySite')
#     print(item['name'], name['zh-tw'])

    pickData['id'] = int(item['id'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']
    
    pickData['filename'] = item['content']['filename']
    pickData['birthday'] = item['content']['birthday']
    pickData['catchphrase'] = en2twFn(item['content']['catchphrase'])['zh-tw']
    pickData['colors'] = [en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in item['content']['colors']]
    pickData['favoriteSong'] = en2twFn(item['content']['favoriteSong'])['zh-tw']
    pickData['gender'] = en2twFn(item['content']['gender'], True)['zh-tw']
    
    pickData['hobby'] = en2twFn(item['content']['hobby'], True)['zh-tw']
    pickData['personality'] = en2twFn(item['content']['personality'], True)['zh-tw']

    pickData['species'] = en2twFn(item['content']['species'], True)['zh-tw']

    pickData['houseImage'] = item['content']['houseImage'].split('/')[-1].replace('.png', '')
    
    pickData['wallpaper'] = {
        'name': en2twFn(item['content']['wallpaper'])['zh-tw'], 
        'filename': name2fileNameMap[item['content']['wallpaper'][0].upper() + item['content']['wallpaper'][1:]]
    }
    pickData['flooring'] = {
        'name': en2twFn(item['content']['flooring'])['zh-tw'], 
        'filename': name2fileNameMap[item['content']['flooring'][0].upper() + item['content']['flooring'][1:]]
    }
    pickData['furnitureList'] = [idMap[int(furniture)] for furniture in item['content']['furnitureList']]
    
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])

dictName = ['id', 'name_e', 'name_j', 'name_c', 'filename', 'birthday', 'catchphrase', 'colors', 'favoriteSong']
dictName += ['gender', 'hobby', 'personality', 'species', 'houseImage', 'wallpaper', 'flooring', 'furnitureList']

saveFile('villagers', output, dictName)

In [45]:
jpNameMap = {}
url = 'https://gamepedia.jp/ac-switch/archives/28527'
r = requests.get(url)

soup = BeautifulSoup(r.text, 'html.parser')

table = soup.find("table", class_="data-table table-datatable locale-compare-table")
# print(table)

# print(table.prettify())

for tr in table.find("tbody").find_all("tr"):
    tds = tr.find_all('td')
    name = tds[0].find('div').string

    img = tds[1].find('img')
    imgSrc = None
    if img:
        imgSrc = img.get('src').split('?')[0]
    
#     print(name, imgSrc)
    jpNameMap[name.replace('1', '１').replace('2', '２').replace('3', '３').replace('4', '４')] = imgSrc

print('get {} jp name'.format(len(jpNameMap.keys())))

get 391 jp name


In [48]:
villagers = getItem('Villagers')
limit = 0

# https://gamepedia.jp/ac-switch/archives/28527
# https://wiki.biligame.com/dongsen/%E9%98%B3%E6%98%8E

for item in villagers:
    if item['category'] != "Villagers":
        continue
    
#     limit += 1
#     if limit > 10:
#         break
    
    name = en2twFn(item['name'][0].upper() + item['name'][1:], 'onlySite')
#     print(name['zh-tw'])
    
    name['zh-cn']
    imageUrl_jp = jpNameMap[name['ja-jp']]

    wikiUrl = 'https://animalcrossing.fandom.com/wiki/{}'.format(name['en-us'])
    
    r = requests.get(wikiUrl)
    text = r.text
    
    if imageUrl_jp:
        r_ = requests.get(imageUrl_jp, stream=True)
        if r_.status_code == 200:
            with open('../img/villager/{}_interior.png'.format(name['en-us']), 'wb') as f:
                for chunk in r_:
                    f.write(chunk)
    else:
        m = re.search('https://vignette\.wikia\.nocookie\.net/animalcrossing/images/.*\n.*\n.*\n.*\n<td><i><a href=\"/wiki/Animal_Crossing:_New_Horizons\" title=\"Animal Crossing: New Horizons\">New Horizons</a></i> \(interior\)', text)
        if m:
            mm = re.search('https://vignette\.wikia\.nocookie\.net/animalcrossing/images/.*?g/', m.group())
            if mm:
                interiorUrl = mm.group()[:-1]
                r_ = requests.get(interiorUrl, stream=True)
                if r_.status_code == 200:
                    with open('../img/villager/{}_interior.png'.format(name['en-us']), 'wb') as f:
                        for chunk in r_:
                            f.write(chunk)
        else:
            print('-------{} lose house image'.format(name['zh-tw']))
    
    
    m = re.search('<a href=\"(.*.png).*\n.*\n.*\n.*\n.*\n.*\n.*<figcaption class=\"pi-item-spacing pi-caption\">.*</figcaption>', text)
    if m:
        mm = re.search('https://vignette\.wikia\.nocookie\.net/animalcrossing/images/.*.png', m.group())
        if mm:
            imageUrl = mm.group()
            
            r_ = requests.get(imageUrl, stream=True)
            if r_.status_code == 200:
                with open('../img/villager/{}.png'.format(name['en-us']), 'wb') as f:
                    for chunk in r_:
                        f.write(chunk)
    else:
        print('-------{} lose body image'.format(name['zh-tw']))

get Villagers 1234
-------艾栗絲 lose house image
-------艾栗絲 lose body image
-------蘋果 lose body image
-------貴妃 lose house image
-------貴妃 lose body image
-------花娜 lose body image
-------朱古莉 lose house image
-------太平 lose house image
-------歐若拉 lose body image
-------巧蔻 lose body image
-------阿妹 lose body image
-------鳳尾 lose body image
-------黃金雞 lose house image
-------拉拉米 lose body image
-------李克 lose house image
-------甘油 lose body image
-------小嵐 lose house image
-------小啾 lose house image


In [19]:
output = []
reactions = getItem('Reactions')

for item in reactions:

#     print(item['name'])
#     print('')
    pickData = {'type': 'reactions'}
    name = en2twFn(item['name'])

    pickData['name_c'] = name['zh-tw']
    pickData['filename'] = item['content']['internalID']
    pickData['obtainedFrom'] = ''.join([en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in item['content']['obtainedFrom']])

    output.append(pickData)

# output = sorted(output, key=lambda k: k['id'])
dictName = ['name_c', 'filename', 'obtainedFrom']

saveFile('reactions', output, dictName)

get Reactions 44
save reactions over


In [10]:
en2twFn = getEn2twFn()
output = []
clothes = getItem('Clothing')

# themes = set()

for item in clothes:
    pickData = {'type': 'clothes'}
    name = en2twFn(item['name'])
    
    print(name['zh-tw'])
    
    pickData['id'] = int(item['id'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp'] if 'ja-jp' in name else ''
    pickData['name_e'] = name['en-us'] if 'en-us' in name else ''
    
    pickData['category'] = en2twFn(item['content']['category'])['zh-tw']
    pickData['filename'] = item['content']['filename']
    pickData['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickData['sell'] = item['content']['sell']
    pickData['buy'] = item['content']['buy'] if 'buy' in item['content'] else ''
    
    pickData['seasonalAvailability'] = en2twFn(item['content']['seasonalAvailability'])['zh-tw']
    pickData['styles'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['styles']])
    pickData['themes'] = [en2twFn(item)['zh-tw'] for item in item['content']['themes']]
    pickData['villagerEquippable'] = item['content']['villagerEquippable']
    
    if 'variations' in item:
        pickData['variations'] = len(item['variations'])
    else:
        pickData['variations'] = 0
        
        
#     for t in item['content']['themes']:
#         themes.add(t)

    output.append(pickData)
    
output = sorted(output, key=lambda k: k['id'])
dictName = ['name_c', 'name_j', 'name_e', 'category', 'filename', 'obtainedFrom', 'sell', 'buy', 'seasonalAvailability', 'styles', 'themes', 'villagerEquippable', 'variations']

saveFile('clothes', output, dictName)

# for t in themes:
#     print(t)

get 6047 i18n data
Custom 1200 i18n data
get Clothing 1161
皮裙
青蛙頭套
橫條紋T恤
無袖有領襯衫
菱格紋毛衣
盛裝和服
摩斯大衣
工作服
MA-1防風裙
長版襯衫
花紋圖案細肩帶背心
綿羊頭套
騎士皮外套
美式足球安全帽
妖精服
凱絲帽子
大圍巾T恤
漆彈面罩
泰拳褲
絲質小禮帽
七分褲
荷葉邊棉褲
餐館制服
鮮艷髮飾
籃球衣
燈芯絨裙
運動褲
星星髮飾
貴族大衣
風衣
骷髏T恤
雙樑眼鏡
無肩帶背心
蓬蓬裙
樂福鞋
圍裙連袖套
花紋T恤
針織帽
龐克裙褲
運動涼鞋
無袖連帽上衣
籃球鞋
油布外套
連身工作服搭袖套
琥珀眼鏡
絨毛外套
扣帶鞋
刷毛外套
嘻哈風衣服
檸檬裙子
拼布大衣
夢幻色系格子連身裙
扣帶低跟鞋
號碼衣
西裝裙
生日墨鏡
牛仔吊帶褲
植物圖案T恤
阿拉伯服
馬德拉斯格紋襯衫
絨面靴
喇叭褲
荷葉邊褲子
民族風刺繡罩衫
雪花毛衣
晚禮服洋裝
保齡球衫
瓜地馬拉傳統裙子
綿羊服
鬥牛犬運動服
下雪毛衣
針織褲
針織裙
商務皮鞋
迷你墨鏡
柔道服
毛球報童帽
學生帽
咖啡店制服
連帽連身裙
清爽五分褲
韓服裙
高筒運動鞋
皮風衣
迷彩衣服
軍師服
雪國服
單車帽
籃球褲
長版襯衫連身裙
針織罩衫&襯衫
彩虹針織衫
學院風針織罩衫
花紋裙子
公主假髮
亮片刺繡無袖上衣
護士帽
健身坦克背心
西裝外套
安全帽
橡膠圍裙
馬丁靴
動力頭盔
亮面西裝褲
肌肉裝
尖頭靴
拼布鬱金香帽
美式足球褲
絨毛連帽上衣
農作遮陽帽
運動短褲
騎士衣服
阿拉伯公主服
全身緊身衣
網帽
格紋圍巾配毛衣
寬邊草帽
羅馬涼鞋
雪國洋裝
羊毛混紡牛角釦大衣
貴族風洋裝
廚師衣服
動物拖鞋
搞笑藝人亮片褲
花紋蕾絲裙
莫卡辛靴
粗橫紋T恤
步行鞋
簡樸衣服
凱絲五分褲
水手襯衫
凱絲低跟鞋
雕花紳士鞋
麂皮運動鞋
連帽上衣配外套
交叉V領綁帶連身裙
翼紋紳士鞋
尖頭鞋
木鞋
制服褲
Polo衫
法蘭絨格紋熱褲
水涼鞋
奇馬約圖騰背心
蕾絲連身裙
男性簡便和服
貓咪服
室內拖鞋
粗橫紋坦克背心
忍者服
麂皮雪靴
騎師服
護士連身裙
原始人服
摩登少女連身裙
格紋背心裙
毛毛靴
巫女服
圍裙連裙子
動物花紋裙
莫卡辛鞋
迷彩裙
無領襯衫
交叉帶涼鞋
蒸汽龐克護目鏡
蕾絲裙
凱絲運動鞋
毛球毛衣
荷蘭洋裝
撞色有領襯衫
豹紋低跟

In [12]:
output = []
clothes = getItem('Home%20Style')

for item in clothes:
    pickData = {'type': 'homeStyle'}
    name = en2twFn(item['name'])
    
    pickData['id'] = int(item['id'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp'] if 'ja-jp' in name else ''
    pickData['name_e'] = name['en-us'] if 'en-us' in name else ''
    
    pickData['category'] = en2twFn(item['content']['category'])['zh-tw']
    pickData['filename'] = item['content']['filename']
    pickData['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickData['sell'] = item['content']['sell']
    pickData['buy'] = item['content']['buy'] if 'buy' in item['content'] else ''
    pickData['DIY'] = item['content']['dIY']
    
    pickData['tag'] = en2twFn(item['content']['tag'])['zh-tw']
    pickData['themes'] = [en2twFn(theme)['zh-tw'] for theme in item['content']['themes'] if not theme in ['None', 'none']]
    
    if len(item['variations']) > 0:
        print(name['zh-tw'])
    
    output.append(pickData)
    
output = sorted(output, key=lambda k: k['id'])
dictName = ['name_c', 'name_j', 'name_e', 'category', 'filename', 'obtainedFrom', 'sell', 'buy', 'tag', 'themes']

saveFile('homeStyle', output, dictName)

# for t in themes:
#     print(t)

get Home%20Style 561
save homeStyle over


In [14]:
output = []
equippables = getItem('Equippables')

for item in equippables:
    pickData = {'type': 'equippables'}
    name = en2twFn(item['name'])
    
    pickData['id'] = int(item['id'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp'] if 'ja-jp' in name else ''
    pickData['name_e'] = name['en-us'] if 'en-us' in name else ''
    
    pickData['category'] = en2twFn(item['content']['category'])['zh-tw']
    pickData['filename'] = item['content']['filename']
    pickData['obtainedFrom'] = [en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']]
    pickData['sell'] = item['content']['sell']
    pickData['buy'] = item['content']['buy'] if 'buy' in item['content'] else ''
    
    pickData['DIY'] = item['content']['dIY']
    
    if pickData['category'] == '工具' and pickData['name_c'][-1] != '棒':
        continue

    output.append(pickData)
    
output = sorted(output, key=lambda k: k['id'])
dictName = ['name_c', 'name_j', 'name_e', 'category', 'filename', 'obtainedFrom', 'sell', 'buy']

saveFile('equippables', output, dictName)

get Equippables 131
save equippables over


In [25]:
outputFish = []
outputInsects = []
equippables = getItem('Museum')

for item in equippables:
    
    if not item['category'] in ["Fish", "Insects"]:
        continue
    
    if item['category'] == "Fish":
        pickData = {'type': item['category'].lower()}
        name = en2twFn(item['name'])

        pickData['id'] = int(item['content']['critterId'])
        pickData['name_c'] = name['zh-tw']
        pickData['name_j'] = name['ja-jp'] if 'ja-jp' in name else ''
        pickData['name_e'] = name['en-us'] if 'en-us' in name else ''

        pickData['category'] = item['content']['category']
        pickData['filename'] = item['content']['iconFilename']
        pickData['sell'] = item['content']['sell']
        pickData['obtainedFrom'] = item['content']['obtainedFrom']
        pickData['weather'] = item['content']['increasedWeatherChance']
        
        pickData['shadow'] = item['content']['shadow']
        pickData['activeMonths'] = item['content']['activeMonths']
        pickData['catchUnlockCount'] = item['content']['catchUnlockCount']
        pickData['spawnRates'] = item['content']['spawnRates']

        outputFish.append(pickData)

    if item['category'] == "Insects":
        pickData = {'type': item['category'].lower()}
        name = en2twFn(item['name'])

        pickData['id'] = int(item['content']['critterId'])
        pickData['name_c'] = name['zh-tw']
        pickData['name_j'] = name['ja-jp'] if 'ja-jp' in name else ''
        pickData['name_e'] = name['en-us'] if 'en-us' in name else ''

        pickData['category'] = item['content']['category']
        pickData['filename'] = item['content']['iconFilename']
        pickData['sell'] = item['content']['sell']
        pickData['obtainedFrom'] = item['content']['obtainedFrom']
        pickData['weather'] = item['content']['weather']
        
        pickData['activeMonths'] = item['content']['activeMonths']
        pickData['catchUnlockCount'] = item['content']['catchUnlockCount']
        pickData['spawnRates'] = item['content']['spawnRates']

        outputInsects.append(pickData)

outputFish = sorted(outputFish, key=lambda k: k['id'])
dictName = ['id', 'name_c', 'name_j', 'name_e', 'category', 'filename', 'sell', 'obtainedFrom', 'weather', 'shadow', 'catchUnlockCount', 'spawnRates', 'activeMonths']

saveFile('fish', outputFish, dictName)

outputInsects = sorted(outputInsects, key=lambda k: k['id'])
dictName = ['id', 'name_c', 'name_j', 'name_e', 'category', 'filename', 'sell', 'obtainedFrom', 'weather', 'catchUnlockCount', 'spawnRates', 'activeMonths']

saveFile('insects', outputInsects, dictName)

get Museum 303
save fish over
save insects over
